<a href="https://colab.research.google.com/github/nrohrbach/sharedmobility_monitoring/blob/main/sharedmobility_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sharedmobility.ch - Validation of GBFS-Feeds

In [2]:
#pip install datetime

In [29]:
import requests
import pandas as pd
from datetime import datetime, timezone
import json
import pytz

In [4]:
# GBFS Feeds abfragen
url = 'https://gbfs.prod.sharedmobility.ch/v2/gbfs'
feeds = requests.get(url).json()

In [5]:
# Alle GBFS Feeds als Dataframe laden
id = [s['id'] for s in feeds['systems']]
url = [s['url'] for s in feeds['systems']]
gbfsfeeds = pd.DataFrame(list(zip(id, url)),columns =['provider', 'gbfsurl'])
providers = list(gbfsfeeds['provider'])

In [6]:
# Nur einen Provider temporär einstellen!
providers = ['bolt_zurich','velospot','carvelo2go','bird-biel']
providers

['bolt_zurich', 'velospot', 'carvelo2go', 'bird-biel']

In [7]:
# Validieren!
JsonResponse = []
providerList = []
#HasErrors = []
#NumberOfErrors = []

for provider in providers:
    try:
      headers = {
          'accept': '*/*',
          'Content-Type': 'application/json',
          'User-Agent': 'geoinformation@bfe.admin.ch',
      }

      json_data = {
          'url': 'https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ provider +'/gbfs',
          'options': {
              'freefloating': False,
              'docked': False,
              'version': None,
              'auth': {
                  'type': None,
                  'basicAuth': {
                      'user': None,
                      'password': None,
                  },
                  'bearerToken': {
                      'token': None,
                  },
                  'oauthClientCredentialsGrant': {
                      'user': None,
                      'password': None,
                      'tokenUrl': None,
                  },
              },
          },
      }
      response = requests.post('https://gbfs-validator.netlify.app/.netlify/functions/validator', headers=headers, json=json_data).json()

      #providerList.append(provider)
      JsonResponse.append(response)
      #HasErrors.append(response['summary']['hasErrors'])
      #NumberOfErrors.append(response['summary']['errorsCount'])

    except:
      #providerList2.append(provider)
      JsonResponse.append(response)
      #HasErrors.append('nan')
      #NumberOfErrors.append('nan')

In [19]:
ValidationResults = pd.json_normalize(JsonResponse)

In [20]:
ValidationResults['provider'] = providers

In [21]:
ValidationResults

,files,summary.validatorVersion,summary.version.detected,summary.version.validated,summary.hasErrors,summary.errorsCount,provider
0,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,bolt_zurich
1,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,True,13,velospot
2,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,carvelo2go
3,[{'schema': {'$schema': 'http://json-schema.or...,1.0.0,2.3,2.3,False,0,bird-biel


In [22]:
ValidationResults['Validator'] = 'https://gbfs-validator.netlify.app/validator?url=https://gbfs.prod.sharedmobility.ch/v2/gbfs/'+ ValidationResults['provider']+'/gbfs'

In [32]:
ValidationDatum = datetime.today()
ValidationDatum = ValidationDatum.astimezone(pytz.timezone('Europe/Berlin')).strftime("%Y-%m-%d %H:%M" )

ValidationDatum


'2023-08-28 08:19'

In [11]:
ValidationResults = ValidationResults.drop(columns=['files','summary.validatorVersion','summary.version.detected'])

In [12]:
ValidationResults

,summary.version.validated,summary.hasErrors,summary.errorsCount,provider
0,2.3,False,0,bolt_zurich
1,2.3,True,13,velospot
2,2.3,False,0,carvelo2go
3,2.3,False,0,bird-biel


In [ ]:
ValidationResults.to_csv("GBFS_validated.csv",header=True,index=False)

# Create Statusbadges für Übersicht auf Github

In [ ]:
# Funktion die den Farbcode zuweist.
def CreateBadgeColour(row):
  if row['summary.hasErrors']:
    return "red"
  else:
    return "green"

In [ ]:
ValidationResults['Colour']= ValidationResults.apply(lambda row: CreateBadgeColour(row), axis=1)

In [ ]:
ValidationResults['Badge']= '- [![Dokumentation](https://badgen.net/badge/' + ValidationResults['provider'] + '/' + ValidationResults['summary.errorsCount'].astype(str) + '%20errors/' + ValidationResults['Colour'] + '?icon=github)](' + ValidationResults['Validator'] + ')'

In [ ]:
ValidationResults = ValidationResults.rename(columns={'Badge': '# Validation of all GBFS-Feeds'})

In [ ]:
ValidationResults.to_csv('Validation.md', columns= ['# Validation of all GBFS-Feeds'], header=True, index=False)